In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import json
from scipy.stats import pearsonr
import geopandas as gpd

In [ ]:
# Compute pearson correlation coefficient (R) averaged over space (lat and lon) and months for each lead time (i.e. timestep)

In [ ]:
# Function to subset to subpolar and polar regions
def subset_polar_regions(ds):
    return ds.where((ds.latitude >= 60) | (ds.latitude <= -60), drop=True)

def subset_northern_temperate(ds):
    """Subset dataset to the Northern Temperate Zone: 35°N to 60°N."""
    return ds.where((ds.latitude >= 35) & (ds.latitude <= 60), drop=True)

def subset_southern_temperate(ds):
    """Subset dataset to the Northern Temperate Zone: 35°N to 60°N."""
    return ds.where((ds.latitude <= -35) & (ds.latitude >= -60), drop=True)

def subset_tropics(ds):
    """Subset dataset to the Tropics: -23.5° to 23.5° latitude."""
    return ds.where((ds.latitude >= -23.5) & (ds.latitude <= 23.5), drop=True)

def subset_subtropics(ds):
    """
    Subset dataset to the subtropics:
    - Northern Subtropics: 23.5°N to 35°N
    - Southern Subtropics: 23.5°S to 35°S
    """
    return ds.where(
        ((ds.latitude >= 23.5) & (ds.latitude <= 35)) | 
        ((ds.latitude <= -23.5) & (ds.latitude >= -35)),
        drop=True
    )

def subset_africa(ds, africa_gdf):
    """
    Subset an xarray dataset to the Africa region using a GeoDataFrame polygon.
    """
    ds_rio = ds.rio.write_crs("EPSG:4326", inplace=False)
    return ds_rio.rio.clip(africa_gdf.geometry, africa_gdf.crs, drop=True)

africa_gdf = gpd.read_file("Africa_outline.geojson").to_crs("EPSG:4326")

# Preprocess function for unit conversion
def preprocess(ds):
    if "t2m" in ds:
        ds["t2m"] = ds["t2m"] - 273.15
        ds["t2m"].attrs["units"] = "Celsius"
    if "msl" in ds:
        ds["msl"] = ds["msl"] / 100.0
        ds["msl"].attrs["units"] = "hPa"
    if "tp" in ds:
        ds = ds.drop_vars("tp")
    return ds

In [ ]:
# Compute R of IFS-HRES (apply needed subset function depending on required geographic region)

In [ ]:
# Root directory where all monthly folders are stored
base_dir = "../Surface Variables/"

# Define monthly date range and variables
months = pd.date_range("2024-01-01", "2024-12-01", freq="MS")
variables = ["u10", "v10", "t2m", "msl", "q"]


# Collect monthly datasets of Pearson R per timestep
monthly_r_datasets = []

for month in months:
    folder_str = month.strftime("%Y%m01")
    forecast_path = os.path.join(base_dir, folder_str, f"{folder_str}_marsfc_sv_q.nc")
    truth_path = os.path.join(base_dir, folder_str, f"{folder_str}_era5_fc_sv_q.nc")

    print(f"Start for {folder_str}")

    try:
        fc = preprocess(xr.open_dataset(forecast_path))
        era5 = preprocess(xr.open_dataset(truth_path))
    except FileNotFoundError:
        print(f"Missing data for {folder_str}, skipping...")
        continue
    
    era5 = era5.rename({"valid_time": "time"})

    # assign numbers for timesteps instead of datetime stamps
    fc = fc.assign_coords(time=np.arange(len(fc.time)).astype("float64"))
    era5 = era5.assign_coords(time=np.arange(len(era5.time)).astype("float64"))
    print(f"Preprocessing done for {folder_str}")

    # Subset to southern temperate
    fc = subset_southern_temperate(fc)
    era5 = subset_southern_temperate(era5)
    print(f"subset computed for {folder_str}")

    r_dict = {}

    for var in variables:
        fc_var, era5_var = xr.align(fc[var], era5[var])  # ensure same coords

        # Loop over each timestep
        r_list = []
        for t in range(fc_var.sizes["time"]):
            fc_t = fc_var.isel(time=t).values.flatten()
            era5_t = era5_var.isel(time=t).values.flatten()

            mask = np.isfinite(fc_t) & np.isfinite(era5_t)
            if np.any(mask):
                r, _ = pearsonr(fc_t[mask], era5_t[mask])
            else:
                r = np.nan
            r_list.append(r)

        r_dict[var] = (["time"], r_list)

    # Create dataset for this month
    r_ds = xr.Dataset(
        r_dict,
        coords={"time": fc["time"], "month": [month.month]}
    )

    monthly_r_datasets.append(r_ds)
    print(f"Computed Pearson R per timestep for {folder_str}")

# Concatenate all months
annual_r_ds = xr.concat(monthly_r_datasets, dim="month")

# Save the result
annual_r_ds.to_netcdf("SouthernTemperate_marsfc_PearsonR_leadtimes.nc")

In [ ]:
# Compute correlation coefficient across whole year per lead time for IFS-HRES

In [ ]:
# Root directory and variable setup
base_dir = "../Surface Variables/"
months = pd.date_range("2024-01-01", "2024-12-01", freq="MS")
variables = ["u10", "v10", "t2m", "msl", "q"]


# For storing full-year forecast and truth data
year_fc = {var: [] for var in variables}
year_era5 = {var: [] for var in variables}

# Load and store all monthly data
for month in months:
    folder_str = month.strftime("%Y%m01")
    forecast_path = os.path.join(base_dir, folder_str, f"{folder_str}_marsfc_sv_q.nc")
    truth_path = os.path.join(base_dir, folder_str, f"{folder_str}_era5_fc_sv_q.nc")
    print(f"Starting for {folder_str}")

    fc = preprocess(xr.open_dataset(forecast_path))
    era5 = preprocess(xr.open_dataset(truth_path))
    era5 = era5.rename({"valid_time": "time"})

    # Ensure numeric time for consistency
    fc = fc.assign_coords(time=np.arange(len(fc.time)).astype("float64"))
    era5 = era5.assign_coords(time=np.arange(len(era5.time)).astype("float64"))

    # Subset to southern temperate
    fc = subset_southern_temperate(fc)
    era5 = subset_southern_temperate(era5)
    print(f"subset computed for {folder_str}")

    # Store monthly slices for each variable
    for var in variables:
        year_fc[var].append(fc[var])
        year_era5[var].append(era5[var])

    print(f"Stored raw data for {folder_str}")

# Compute Pearson R per timestep (0–40) across all months (so, 12 * lat * lon per timestep)
print(f"Starting to compute annual Pearson R per timestep...")

full_year_r = {}

for var in variables:
    print(f"Processing {var}...")

    # Concatenate monthly data along a new "month" dimension
    all_fc = xr.concat(year_fc[var], dim="month")
    all_era5 = xr.concat(year_era5[var], dim="month")

    r_list = []

    for t in range(all_fc.sizes["time"]):  # Loop over 41 timesteps
        # Extract data for timestep t across all months, then flatten spatially
        fc_t = all_fc.isel(time=t).values.reshape(12, -1)  # (month, space)
        era5_t = all_era5.isel(time=t).values.reshape(12, -1)

        # Flatten across all months and space
        fc_flat = fc_t.flatten()
        era5_flat = era5_t.flatten()

        mask = np.isfinite(fc_flat) & np.isfinite(era5_flat)
        if np.any(mask):
            r, _ = pearsonr(fc_flat[mask], era5_flat[mask])
        else:
            r = np.nan

        r_list.append(r)

    full_year_r[var] = (["time"], r_list)

# Final dataset with time (41 timesteps) and month = 13
correlation_annual_ds = xr.Dataset(
    full_year_r,
    coords={"time": np.arange(41), "month": [13]}
)

# Save to NetCDF
correlation_annual_ds.to_netcdf("SouthernTemperate_marsfc_PearsonR_leadtimes_13.nc")
print("Saved to 'SouthernTemperate_marsfc_PearsonR_leadtimes_13.nc'")

In [ ]:
# merge "marsfc_PearsonR_leadtimes_13.nc" with "marsfc_PearsonR_per_timestep.nc"

In [ ]:
annual = xr.open_dataset("SouthernTemperate_marsfc_PearsonR_leadtimes_13.nc")
monthly = xr.open_dataset("SouthernTemperate_marsfc_PearsonR_leadtimes.nc")

monthly_rmse_ds_with_annual = xr.concat([monthly, annual], dim="month")

# Save final dataset
monthly_rmse_ds_with_annual.to_netcdf("SouthernTemperate/PerLeadTime/SouthernTemperate_marsfc_PearsonR_leadtimes_w_annual.nc")

In [ ]:
# Compute R of AIFS (apply needed subset function depending on required geographic region)

In [ ]:
# Root directory where all monthly folders are stored
base_dir = "../Surface Variables/"

# Define monthly date range and variables
months = pd.date_range("2024-03-01", "2024-12-01", freq="MS")
variables = ["u10", "v10", "t2m", "msl", "q"]

# Collect monthly datasets of Pearson R per timestep
monthly_r_datasets = []

for month in months:
    folder_str = month.strftime("%Y%m01")
    forecast_path = os.path.join(base_dir, folder_str, f"{folder_str}_marsai_sv_q.nc")
    truth_path = os.path.join(base_dir, folder_str, f"{folder_str}_era5_gcai_sv_q.nc")

    print(f"Start for {folder_str}")

    try:
        fc = preprocess(xr.open_dataset(forecast_path))
        era5 = preprocess(xr.open_dataset(truth_path))
    except FileNotFoundError:
        print(f"Missing data for {folder_str}, skipping...")
        continue
    
    era5 = era5.rename({"valid_time": "time"})

    # assign numbers for timesteps instead of datetime stamps
    fc = fc.assign_coords(time=np.arange(len(fc.time)).astype("float64"))
    era5 = era5.assign_coords(time=np.arange(len(era5.time)).astype("float64"))

    print(f"Preprocessing done for {folder_str}")

    # Subset to southern temperate
    fc = subset_southern_temperate(fc)
    era5 = subset_southern_temperate(era5)
    print(f"subset computed for {folder_str}")

    r_dict = {}

    for var in variables:
        fc_var, era5_var = xr.align(fc[var], era5[var])  # ensure same coords

        # Loop over each timestep
        r_list = []
        for t in range(fc_var.sizes["time"]):
            fc_t = fc_var.isel(time=t).values.flatten()
            era5_t = era5_var.isel(time=t).values.flatten()

            mask = np.isfinite(fc_t) & np.isfinite(era5_t)
            if np.any(mask):
                r, _ = pearsonr(fc_t[mask], era5_t[mask])
            else:
                r = np.nan
            r_list.append(r)

        r_dict[var] = (["time"], r_list)

    # Create dataset for this month
    r_ds = xr.Dataset(
        r_dict,
        coords={"time": fc["time"], "month": [month.month]}
    )

    monthly_r_datasets.append(r_ds)
    print(f"Computed Pearson R per timestep for {folder_str}")

# Concatenate all months
annual_r_ds = xr.concat(monthly_r_datasets, dim="month")

# Save the result
annual_r_ds.to_netcdf("SouthernTemperate_marsai_PearsonR_leadtimes.nc")
print("Saved to 'SouthernTemperate_marsai_PearsonR_leadtimes.nc'")

In [ ]:
# Compute correlation coefficient across whole year per lead time for AIFS

In [ ]:
# Root directory and variable setup
base_dir = "../Surface Variables/"
months = pd.date_range("2024-03-01", "2024-12-01", freq="MS")
variables = ["u10", "v10", "t2m", "msl", "q"]


# For storing full-year forecast and truth data
year_fc = {var: [] for var in variables}
year_era5 = {var: [] for var in variables}

# Load and store all monthly data
for month in months:
    folder_str = month.strftime("%Y%m01")
    forecast_path = os.path.join(base_dir, folder_str, f"{folder_str}_marsai_sv_q.nc")
    truth_path = os.path.join(base_dir, folder_str, f"{folder_str}_era5_gcai_sv_q.nc")
    print(f"Starting for {folder_str}")

    fc = preprocess(xr.open_dataset(forecast_path))
    era5 = preprocess(xr.open_dataset(truth_path))
    era5 = era5.rename({"valid_time": "time"})

    # Ensure numeric time for consistency
    fc = fc.assign_coords(time=np.arange(len(fc.time)).astype("float64"))
    era5 = era5.assign_coords(time=np.arange(len(era5.time)).astype("float64"))
    print(f"preprocessing done for {folder_str}")

    # Subset to southern temperate
    fc = subset_southern_temperate(fc)
    era5 = subset_southern_temperate(era5)
    print(f"subset computed for {folder_str}")

    # Store monthly slices for each variable
    for var in variables:
        year_fc[var].append(fc[var])
        year_era5[var].append(era5[var])

    print(f"Stored raw data for {folder_str}")

# Compute Pearson R per timestep (0–40) across all months (so, 12 * lat * lon per timestep)
print(f"Starting to compute annual Pearson R per timestep...")

full_year_r = {}

for var in variables:
    print(f"Processing {var}...")

    # Concatenate monthly data along a new "month" dimension
    all_fc = xr.concat(year_fc[var], dim="month")
    all_era5 = xr.concat(year_era5[var], dim="month")

    r_list = []

    for t in range(all_fc.sizes["time"]):  # Loop over 41 timesteps
        # Extract data for timestep t across all months, then flatten spatially
        fc_t = all_fc.isel(time=t).values.reshape(10, -1)  # (month, space)
        era5_t = all_era5.isel(time=t).values.reshape(10, -1)

        # Flatten across all months and space
        fc_flat = fc_t.flatten()
        era5_flat = era5_t.flatten()

        mask = np.isfinite(fc_flat) & np.isfinite(era5_flat)
        if np.any(mask):
            r, _ = pearsonr(fc_flat[mask], era5_flat[mask])
        else:
            r = np.nan

        r_list.append(r)

    full_year_r[var] = (["time"], r_list)

# Final dataset with time (41 timesteps) and month = 13
correlation_annual_ds = xr.Dataset(
    full_year_r,
    coords={"time": np.arange(41), "month": [13]}
)

# Save to NetCDF
correlation_annual_ds.to_netcdf("SouthernTemperate_marsai_PearsonR_leadtimes_13.nc")
print("Saved to 'SouthernTemperate_marsai_PearsonR_leadtimes_13.nc'")

In [ ]:
# merge "marsai_PearsonR_leadtimes.nc" with "marsai_PearsonR_leadtimes_13.nc"

In [ ]:
annual = xr.open_dataset("SouthernTemperate_marsai_PearsonR_leadtimes_13.nc")
monthly = xr.open_dataset("SouthernTemperate_marsai_PearsonR_leadtimes.nc")

monthly_rmse_ds_with_annual = xr.concat([monthly, annual], dim="month")

# Save final dataset
monthly_rmse_ds_with_annual.to_netcdf("SouthernTemperate/PerLeadTime/SouthernTemperate_marsai_PearsonR_leadtimes_w_annual.nc")

In [ ]:
# Compute R of GraphCast (apply needed subset function depending on required geographic region)

In [ ]:
# Root directory where all monthly folders are stored
base_dir = "../Surface Variables/"

# Define monthly date range and variables
months = pd.date_range("2024-01-01", "2024-12-01", freq="MS")
variables = ["u10", "v10", "t2m", "msl", "q"]


# Collect monthly datasets of Pearson R per timestep
monthly_r_datasets = []

for month in months:
    folder_str = month.strftime("%Y%m01")
    forecast_path = os.path.join(base_dir, folder_str, f"{folder_str}_gc_sv_q.nc")
    truth_path = os.path.join(base_dir, folder_str, f"{folder_str}_era5_gcai_sv_q.nc")

    print(f"Start for {folder_str}")

    try:
        fc = preprocess(xr.open_dataset(forecast_path))
        era5 = preprocess(xr.open_dataset(truth_path))
    except FileNotFoundError:
        print(f"Missing data for {folder_str}, skipping...")
        continue
    
    era5 = era5.rename({"valid_time": "time"})

    # Drop the existing "time" coordinate to avoid conflicts
    if "time" in fc.coords:
        fc = fc.drop_vars("time")

    # Swap the "step" dimension with "valid_time" and rename it to "time"
    fc = fc.swap_dims({"step": "valid_time"})  # make valid_time a dimension
    fc = fc.rename({"valid_time": "time"})     # rename the dimension to "time"

    # assign numbers for timesteps instead of datetime stamps
    fc = fc.assign_coords(time=np.arange(len(fc.time)).astype("float64"))
    era5 = era5.assign_coords(time=np.arange(len(era5.time)).astype("float64"))

    print(f"Preprocessing done for {folder_str}")

    # Subset to southern temperate
    fc = subset_southern_temperate(fc)
    era5 = subset_southern_temperate(era5)
    print(f"subset computed for {folder_str}")

    r_dict = {}

    for var in variables:
        fc_var, era5_var = xr.align(fc[var], era5[var])  # ensure same coords

        # Loop over each timestep
        r_list = []
        for t in range(fc_var.sizes["time"]):
            fc_t = fc_var.isel(time=t).values.flatten()
            era5_t = era5_var.isel(time=t).values.flatten()

            mask = np.isfinite(fc_t) & np.isfinite(era5_t)
            if np.any(mask):
                r, _ = pearsonr(fc_t[mask], era5_t[mask])
            else:
                r = np.nan
            r_list.append(r)

        r_dict[var] = (["time"], r_list)

    # Create dataset for this month
    r_ds = xr.Dataset(
        r_dict,
        coords={"time": fc["time"], "month": [month.month]}
    )

    monthly_r_datasets.append(r_ds)
    print(f"Computed Pearson R per timestep for {folder_str}")

# Concatenate all months
annual_r_ds = xr.concat(monthly_r_datasets, dim="month")

# Save the result
annual_r_ds.to_netcdf("SouthernTemperate_gc_PearsonR_leadtimes.nc")
print("Saved to 'SouthernTemperate_gc_PearsonR_leadtimes.nc'")

In [ ]:
# compute R across whole year per lead time for graphcast

In [ ]:
# Root directory and variable setup
base_dir = "../Surface Variables/"
months = pd.date_range("2024-01-01", "2024-12-01", freq="MS")
variables = ["u10", "v10", "t2m", "msl", "q"]

# For storing full-year forecast and truth data
year_fc = {var: [] for var in variables}
year_era5 = {var: [] for var in variables}

# Load and store all monthly data
for month in months:
    folder_str = month.strftime("%Y%m01")
    forecast_path = os.path.join(base_dir, folder_str, f"{folder_str}_gc_sv_q.nc")
    truth_path = os.path.join(base_dir, folder_str, f"{folder_str}_era5_gcai_sv_q.nc")
    print(f"Starting for {folder_str}")

    fc = preprocess(xr.open_dataset(forecast_path))
    era5 = preprocess(xr.open_dataset(truth_path))
    era5 = era5.rename({"valid_time": "time"})

    # Drop the existing "time" coordinate to avoid conflicts
    if "time" in fc.coords:
        fc = fc.drop_vars("time")

    # Swap the "step" dimension with "valid_time" and rename it to "time"
    fc = fc.swap_dims({"step": "valid_time"})  # make valid_time a dimension
    fc = fc.rename({"valid_time": "time"})     # rename the dimension to "time"

    # Ensure numeric time for consistency
    fc = fc.assign_coords(time=np.arange(len(fc.time)).astype("float64"))
    era5 = era5.assign_coords(time=np.arange(len(era5.time)).astype("float64"))
    print(f"preprocessed for {folder_str}")

    # Subset to southern temperate
    fc = subset_southern_temperate(fc)
    era5 = subset_southern_temperate(era5)
    print(f"subset computed for {folder_str}")

    # Store monthly slices for each variable
    for var in variables:
        year_fc[var].append(fc[var])
        year_era5[var].append(era5[var])

    print(f"Stored raw data for {folder_str}")

# Compute Pearson R per timestep (0–40) across all months (so, 12 * lat * lon per timestep)
print(f"Starting to compute annual Pearson R per timestep...")

full_year_r = {}

for var in variables:
    print(f"Processing {var}...")

    # Concatenate monthly data along a new "month" dimension
    all_fc = xr.concat(year_fc[var], dim="month")
    all_era5 = xr.concat(year_era5[var], dim="month")

    r_list = []

    for t in range(all_fc.sizes["time"]):  # Loop over 41 timesteps
        # Extract data for timestep t across all months, then flatten spatially
        fc_t = all_fc.isel(time=t).values.reshape(12, -1)  # (month, space)
        era5_t = all_era5.isel(time=t).values.reshape(12, -1)

        # Flatten across all months and space
        fc_flat = fc_t.flatten()
        era5_flat = era5_t.flatten()

        mask = np.isfinite(fc_flat) & np.isfinite(era5_flat)
        if np.any(mask):
            r, _ = pearsonr(fc_flat[mask], era5_flat[mask])
        else:
            r = np.nan

        r_list.append(r)

    full_year_r[var] = (["time"], r_list)

# Final dataset with time (41 timesteps) and month = 13
correlation_annual_ds = xr.Dataset(
    full_year_r,
    coords={"time": np.arange(41), "month": [13]}
)

# Save to NetCDF
correlation_annual_ds.to_netcdf("SouthernTemperate_gc_PearsonR_leadtimes_13.nc")
print("Saved to 'SouthernTemperate_gc_PearsonR_leadtimes_13.nc'")
print(correlation_annual_ds)

In [ ]:
# merge "gc_PearsonR_leadtimes.nc" with "gc_PearsonR_leadtimes_13.nc"

In [ ]:
annual = xr.open_dataset("SouthernTemperate_gc_PearsonR_leadtimes_13.nc")
monthly = xr.open_dataset("SouthernTemperate_gc_PearsonR_leadtimes.nc")

monthly_rmse_ds_with_annual = xr.concat([monthly, annual], dim="month")

# Save final dataset
monthly_rmse_ds_with_annual.to_netcdf("SouthernTemperate/PerLeadTime/SouthernTemperate_gc_PearsonR_leadtimes_w_annual.nc")

In [ ]:
# merge all models into 1 dataset

In [ ]:
# Load datasets
air = xr.open_dataset("SouthernTemperate/PerLeadTime/SouthernTemperate_marsai_PearsonR_leadtimes_w_annual.nc")
gcr = xr.open_dataset("SouthernTemperate/PerLeadTime/SouthernTemperate_gc_PearsonR_leadtimes_w_annual.nc")
fcr = xr.open_dataset("SouthernTemperate/PerLeadTime/SouthernTemperate_marsfc_PearsonR_leadtimes_w_annual.nc")

# Ensure all datasets have the full month range 1 to 13
full_months = np.arange(1, 14)

# Reindex to include all months, filling missing with NaN
air = air.reindex(month=full_months)
gcr = gcr.reindex(month=full_months)
fcr = fcr.reindex(month=full_months)

# Drop any unrelated extra coordinates to match structure 
drop_coords = ["meanSea", "surface", "isobaricInhPa", "number", "expver", "step"]
gcr = gcr.drop_vars([c for c in drop_coords if c in gcr])
air = air.drop_vars([c for c in drop_coords if c in air])
fcr = fcr.drop_vars([c for c in drop_coords if c in fcr])

# Stack them into a new 'model' dimension
combined = xr.concat([air, gcr, fcr], dim="model")

# Add model labels
combined = combined.assign_coords(model=["marsai", "gc", "marsfc"])

# Save the merged dataset (optional)
combined.to_netcdf("SouthernTemperate/PerLeadTime/SouthernTemperate_PearsonR_leadtimes_allmodels.nc")

# Print confirmation
print("Merged dataset created with shape:", combined.sizes)
print("Models:", combined.model.values)